# Data Source

In [ ]:
data_input = '/home/tanishq/Desktop/DeepWork/AI/Data/dataset.csv'
test_input = '/home/tanishq/Desktop/DeepWork/AI/Data/test_dataset.csv'

# `Imports`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

plt.style.use("ggplot")
%matplotlib inline

# `Data Exploration`

## Input

In [ ]:
df = pd.read_csv(data_input)
df_test = pd.read_csv(test_input)

In [ ]:
df.tail()

In [ ]:
df_test.tail()

## Details For Training Data

In [ ]:
df.describe()

In [ ]:
print("Number of Sentences: {}".format(len(pd.unique(df["Sentence ID"].values))))
print("Number of Words: {}".format(len(pd.unique(df["Word"].values))))
print("Number of POS Tags: {}".format(len(pd.unique(df["POS Tag"].values))))
print("Number of NE Tags: {}".format(len(pd.unique(df["NER Tag"].values))))
print()
print()
print("List of Tags:")
print(pd.unique(df["POS Tag"].values))
print()
print("List of Entity Tags:")
print(pd.unique(df["NER Tag"].values))

In [ ]:
print("Number of Sentences: {}".format(len(pd.unique(df_test["Sentence ID"].values))))
print("Number of Words: {}".format(len(pd.unique(df_test["Word"].values))))
print("Number of POS Tags: {}".format(len(pd.unique(df_test["POS Tag"].values))))
print("Number of NE Tags: {}".format(len(pd.unique(df_test["NER Tag"].values))))
print()
print()
print("List of Tags:")
print(pd.unique(df_test["POS Tag"].values))
print()
print("List of Entity Tags:")
print(pd.unique(df_test["NER Tag"].values))

In [ ]:
df_test.describe()

## Generating Sentences

In [ ]:
aggregate_function = lambda s: [(w, p, t) for w, p, t in zip(
    list(s['Word'].values),
    list(s['POS Tag'].values),
    list(s['NER Tag'].values)
)]

In [ ]:
sentences = df.groupby("Sentence ID").apply(aggregate_function)
sentences_test = df_test.groupby("Sentence ID").apply(aggregate_function)

## Graphs

In [ ]:
plt.hist([len(s) for s in sentences], bins=50)
plt.show()

max_length = max(len(s) for s in sentences)
print("Maximum Length: ", max_length)

In [ ]:
plt.hist([len(s) for s in sentences_test], bins=50)
plt.show()

max_length_test = max(len(s) for s in sentences_test)
print("Maximum Length: ", max_length_test)

## Preparing Words and Tags for Embeddings

In [ ]:
words = list(set(df["Word"].values))
words.append("__PAD__")
n_words = len(words)
print(n_words)

In [ ]:
words_test = list(set(df_test["Word"].values))
words_test.append("__PAD__")
n_words_test = len(words_test)
print(n_words_test)

In [ ]:
tags = list(set(df["NER Tag"].values))
n_tags = len(tags)
print(n_tags)

In [ ]:
word2id = {w: i for i, w in enumerate(words)}
tag2id = {t: i for i, t in enumerate(tags)}

# `Training & Testing Data`

## Getting Sentences

In [ ]:
X_train_sent = [[tup[0] for i, tup in enumerate(sent) if i<max_length] for sent in sentences]
y_train_sent = [[tup[2] for i, tup in enumerate(sent) if i<max_length] for sent in sentences]

In [ ]:
X_test_sent = [[tup[0] for i, tup in enumerate(sent) if i<max_length] for sent in sentences_test]
y_test_sent = [[tup[2] for i, tup in enumerate(sent) if i<max_length] for sent in sentences_test]

## Generating Encodings

In [ ]:
def generate_encodings(X_sent, y_sent):
    X_train = []
    y_train = []
    
    for x_s, y_s in zip(X_sent, y_sent):

        temp_x = []
        temp_y = []

        for x, y in zip(x_s, y_s):
            try:
                x = word2id[x]
            except:
                x = word2id['__PAD__']
            try:
                y = tag2id[y]
            except:
                y = tag2id['O']

            temp_x.append(x)
            temp_y.append(y)

        X_train.append(temp_x)
        y_train.append(temp_y)

    return (X_train, y_train)

In [ ]:
X_train, y_train = generate_encodings(X_train_sent, y_train_sent)

In [ ]:
X_test, y_test = generate_encodings(X_test_sent, y_test_sent)

## Padding

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', value=word2id['__PAD__'])
y_train = pad_sequences(y_train, maxlen=max_length, padding='post', value=tag2id['O'])

In [ ]:
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', value=word2id['__PAD__'])
y_test = pad_sequences(y_test, maxlen=max_length, padding='post', value=tag2id['O'])

## Convert y to categories

In [ ]:
y_train = [to_categorical(i, n_tags) for i in y_train]

# `Memory Tagging`

In [ ]:
class MemoryTagger(BaseEstimator):
    
    def fit(self, X, y):
        """
        Parameters:
        
        X: a list of words
        eg: ["The", "sky", "is", "blue", "."]
        
        y: a lsit of Entity Tags
        eg: ["O", "B-nat", "I-nat", "O"]
        """
        
        vocab = {}
        self.tags = []
        for x, t in zip(X, y):
            
            # stores the distinct tags used
            if t not in self.tags:
                self.tags.append(t)
            
            # if x is a word encountered before
            if x in vocab:
                # if tag is seen before
                if t in vocab[x]:
                    vocab[x][t] += 1
                else:
                    vocab[x][t] = 0
            # first time encountering the word
            else:
                vocab[x] = {t: 0}
            
        # Finding the most popular tags for the word
        self.memory = {}
        for key, d in vocab.items():
            self.memory[key] = max(d, key=d.get)
    
    def predict(self, X, y=None):
        """
        Predict the the tag from memory. If the word is unknown, predicts 'O'.
        """
        return [self.memory.get(x, 'O') for x in X]

In [ ]:
tagger = MemoryTagger()

In [ ]:
X = list(df["Word"].values)
y = list(df["NER Tag"].values)

In [ ]:
predictions = cross_val_predict(estimator=MemoryTagger(), X=X, y=y, cv=5)

In [ ]:
report = classification_report(y_pred=predictions, y_true=y)
print(report)

# Deep Learning

In [ ]:
model = Sequential([
                    Embedding(input_dim=n_words, output_dim=64, input_length=max_length),
                    Dropout(0.1),
                    Bidirectional(LSTM(
                        128, 
                        activation='tanh', 
                        return_sequences=True, 
                        recurrent_activation='sigmoid', 
                        use_bias=True,
                        )),
                    TimeDistributed(Dense(
                        n_tags,
                        activation='softmax'
                    ))
])

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=3, verbose=1, validation_split=0.4)